In [90]:
#import libraries
import fiona
import pandas as pd
import numpy as np

import os

#bokeh
from bokeh.io import show, output_notebook, push_notebook, curdoc, output_file
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import layout, column, row
from bokeh.models.selections import Selection
from bokeh.models import NumberFormatter,CustomJS, Panel, Spacer,HoverTool,LogColorMapper, ColumnDataSource, TapTool, BoxSelectTool, LabelSet, Label, FactorRange,NumeralTickFormatter
from bokeh.tile_providers import STAMEN_TERRAIN_RETINA,CARTODBPOSITRON_RETINA
from bokeh.core.properties import value
from bokeh.transform import factor_cmap, dodge
from bokeh.models.widgets import Div, Tabs, Paragraph, Dropdown, Button, PreText, Toggle, TableColumn, DataTable

#mapping
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon
import geopandas as gpd

from bokeh.transform import factor_cmap
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.core.properties import value

#color 
from bokeh.palettes import Spectral6

import warnings
warnings.filterwarnings('ignore')
output_notebook()

Loading BokehJS ...

In [91]:
cur = os.getcwd()
ecd = os.path.join(cur,'data','shapefiles','ecd_shp.shp')


#http://tfresource.org/Benefits_of_Activity_Based_Models
#http://www.cmap.illinois.gov/documents/10180/15634/CMAP_CT-RAMP_Final_Report_10-2011.pdf/6d3cba41-f568-4f21-99dd-e4d55c81ad58
#http://atrf.info/papers/2010/2010_Davidson_Vovsha_Freedman_Donnelly.pdf
#https://atlantaregional.org/wp-content/uploads/tp-mug-arcactivitybasedmodeloverviewdeployment-031116.pdf

In [92]:
def make_group_vbar(df, groups, subgroups, tool_tips, chart_tools, p_width = 400, p_height = 200,
                chart_title="Sample Grouped Bar Chart"):

    df_groupby = df.groupby(groups).sum().reset_index()
    df_groups = df_groupby[groups].values.tolist()
    numgroups = len(subgroups)

    data = {'groups': groups}

    ziplist = ()
    for s in subgroups:
        data[s] = df_groupby[s].values.tolist()
        ziplist += (data[s],)


    x = [(g, s) for g in df_groups for s in subgroups]
    sgroups = [s for g in df_groups for s in subgroups]
    pgroups = [g for g in df_groups for s in subgroups]

    if numgroups == 2:
        counts = sum(zip(ziplist[0], ziplist[1]), ())
    elif numgroups ==3:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2]), ())
    elif numgroups ==4:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4]), ())
    elif numgroups ==5:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4], ziplist[5]), ())
    elif numgroups ==6:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4], ziplist[5], ziplist[6]), ())
    elif numgroups ==7:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                         ziplist[5], ziplist[6], ziplist[7]), ())
    elif numgroups ==8:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                         ziplist[5], ziplist[6], ziplist[7], ziplist[8]), ())
    elif numgroups ==9:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                         ziplist[5], ziplist[6], ziplist[7], ziplist[8], ziplist[9]), ())
    elif numgroups ==10:
        counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                         ziplist[5], ziplist[6], ziplist[7], ziplist[8], ziplist[9], ziplist[10]), ())

    source = ColumnDataSource(data=dict(x=x, counts=counts, sub=sgroups, prime=pgroups))


    p = figure(x_range=FactorRange(*x), plot_width = p_width,plot_height=p_height, title=chart_title,
       toolbar_location='right', tools=chart_tools,
       tooltips=tool_tips)

    p.vbar(x='x', top='counts', width=0.9, source=source)


    # Styling
    #p = bar_style(p)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    return p
def make_base_map(tile_map=CARTODBPOSITRON_RETINA,map_width=800,map_height=500, xaxis=None, yaxis=None,
                xrange=(-9990000,-9619944), yrange=(5011119,5310000),plot_tools="pan,wheel_zoom,reset"):

    p = figure(tools=plot_tools, width=map_width,height=map_height, x_axis_location=xaxis, y_axis_location=yaxis,
                x_range=xrange, y_range=yrange)

    p.grid.grid_line_color = None

    p.add_tile(tile_map)

    return p
def make_poly_map(base_map, shapefile,label,fillcolor,fillalpha,linecolor,lineweight,add_label,legend_field):

    p = base_map

    shp = fiona.open(shapefile)

    # Extract features from shapefile
    district_name = [ feat["properties"][label].replace(" County","") for feat in shp]
    pareas = [ feat["properties"][legend_field] for feat in shp]
    pop = [ feat["properties"]["TOT_POP"] for feat in shp]
    district_area = [ feat["properties"]["Shape_Area"] for feat in shp]
    district_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp]
    district_y = [ [y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp]
    district_xy = [ [ xy for xy in feat["geometry"]["coordinates"][0]] for feat in shp]
    district_poly = [ Polygon(xy) for xy in district_xy] # coords to Polygon

    source = ColumnDataSource(data=dict(
        x=district_x, y=district_y,
        name=district_name,
        planning = pareas,
        pop=pop
    ))

    polygons = p.patches('x', 'y', source=source, fill_color=fillcolor,
              fill_alpha=fillalpha, line_color=linecolor, line_width=lineweight, legend=legend_field)

    if add_label:

        labels = LabelSet(x='label_x', y='label_y', source=source,text='name', level='glyph',text_line_height=1.5,
                  x_offset = -15,y_offset = -8,render_mode='canvas',text_font_size="10pt",text_color="white")

        p.add_layout(labels)
        
    TOOLTIPS = [
        ("Census Tract", '@name'),
        ("Total Population", '@pop')
    ]

    p.add_tools(HoverTool(tooltips=TOOLTIPS, renderers=[polygons]))


    return p

In [93]:
p = make_base_map(map_width=400,map_height=500, xaxis=None, yaxis=None,
                    xrange=(-9890000,-9719944), yrange=(5211119,5110000),plot_tools="pan,wheel_zoom,reset,save")

poly_plot = make_poly_map(p, ecd, 'NAME','Blue',.5,'White',.1,False,"EDA_FLAG")

#show(poly_plot)

ERROR:Fiona:CPLE_OpenFailed in b'Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.'


In [94]:
#ABM Overview
def intro_tab(rings, rings_pts, metra, cta, counties, mhn):

    column_width = 1400
    bar_height = 500
    census_color = "#EFF1EF"
    survey_color = '#9EA499'
    cmap_color = '#495667'

    def make_group_vbar(df, groups, subgroups, tool_tips, chart_tools, p_width = 400, p_height = 200,
                    chart_title="Sample Grouped Bar Chart"):

        df_groupby = df.groupby(groups).sum().reset_index()
        df_groups = df_groupby[groups].values.tolist()
        numgroups = len(subgroups)

        data = {'groups': groups}

        ziplist = ()
        for s in subgroups:
            data[s] = df_groupby[s].values.tolist()
            ziplist += (data[s],)


        x = [(g, s) for g in df_groups for s in subgroups]
        sgroups = [s for g in df_groups for s in subgroups]
        pgroups = [g for g in df_groups for s in subgroups]

        if numgroups == 2:
            counts = sum(zip(ziplist[0], ziplist[1]), ())
        elif numgroups ==3:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2]), ())
        elif numgroups ==4:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4]), ())
        elif numgroups ==5:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4], ziplist[5]), ())
        elif numgroups ==6:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4], ziplist[5], ziplist[6]), ())
        elif numgroups ==7:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                             ziplist[5], ziplist[6], ziplist[7]), ())
        elif numgroups ==8:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                             ziplist[5], ziplist[6], ziplist[7], ziplist[8]), ())
        elif numgroups ==9:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                             ziplist[5], ziplist[6], ziplist[7], ziplist[8], ziplist[9]), ())
        elif numgroups ==10:
            counts = sum(zip(ziplist[0], ziplist[1], ziplist[2], ziplist[3], ziplist[4],
                             ziplist[5], ziplist[6], ziplist[7], ziplist[8], ziplist[9], ziplist[10]), ())

        source = ColumnDataSource(data=dict(x=x, counts=counts, sub=sgroups, prime=pgroups))


        p = figure(x_range=FactorRange(*x), plot_width = p_width,plot_height=p_height, title=chart_title,
           toolbar_location='right', tools=chart_tools,
           tooltips=tool_tips)

        p.vbar(x='x', top='counts', width=0.9, source=source)


        # Styling
        #p = bar_style(p)

        p.y_range.start = 0
        p.x_range.range_padding = 0.1
        p.xaxis.major_label_orientation = 1
        p.xgrid.grid_line_color = None

        return p
    def make_base_map(tile_map=CARTODBPOSITRON_RETINA,map_width=800,map_height=500, xaxis=None, yaxis=None,
                    xrange=(-9990000,-9619944), yrange=(5011119,5310000),plot_tools="pan,wheel_zoom,reset"):

        p = figure(tools=plot_tools, width=map_width,height=map_height, x_axis_location=xaxis, y_axis_location=yaxis,
                    x_range=xrange, y_range=yrange)

        p.grid.grid_line_color = None

        p.add_tile(tile_map)

        return p
    def make_poly_map(base_map, shapefile,label,fillcolor,fillalpha,linecolor,lineweight,add_label,legend_field):

        p = base_map

        shp = fiona.open(shapefile)

        # Extract features from shapefile
        district_name = [ feat["properties"][label].replace(" County","") for feat in shp]
        fill_color = [ feat["properties"]["color"] for feat in shp]
        pareas = [ feat["properties"]["legend"] for feat in shp]
        label_x = [ feat["properties"]["INSIDE_X"] for feat in shp]
        label_y = [ feat["properties"]["INSIDE_Y"] for feat in shp]
        district_area = [ feat["properties"]["Shape_Area"] for feat in shp]
        district_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp]
        district_y = [ [y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp]
        district_xy = [ [ xy for xy in feat["geometry"]["coordinates"][0]] for feat in shp]
        district_poly = [ Polygon(xy) for xy in district_xy] # coords to Polygon

        source = ColumnDataSource(data=dict(
            x=district_x, y=district_y,
            name=district_name,
            poly_color=fill_color,
            planning = pareas,
            label_x = label_x,
            label_y = label_y
        ))

        polygons = p.patches('x', 'y', source=source, fill_color=fillcolor,
                  fill_alpha=fillalpha, line_color=linecolor, line_width=lineweight, legend=legend_field)

        if add_label:

            labels = LabelSet(x='label_x', y='label_y', source=source,text='name', level='glyph',text_line_height=1.5,
                      x_offset = -15,y_offset = -8,render_mode='canvas',text_font_size="10pt",text_color="white")

            p.add_layout(labels)
        #TOOLTIPS = [
        #    ("Count", '@rs2'),
        #    ("Total Transit Trips From Sector", "@alightings_s{0,0}"),
        #    ("Total Transit Trips To Sector", "@boardings_s{0,0}")
        #]

        p.add_tools(HoverTool(renderers=[polygons]))


        return p
    def make_line_map(base_map, shp):

        p = base_map

        def getLineCoords(row, geom, coord_type):
            """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
            if coord_type == 'x':
                return list( row[geom].coords.xy[0] )
            elif coord_type == 'y':
                return list( row[geom].coords.xy[1] )

        gpd_shp = gpd.read_file(shp)

        gpd_shp['x'] = gpd_shp.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)

        # Calculate y coordinates of the line
        gpd_shp['y'] = gpd_shp.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

        # Make a copy and drop the geometry column
        shp_df = gpd_shp.drop('geometry', axis=1).copy()

        # Point DataSource
        source = ColumnDataSource(shp_df)

        p.multi_line('x', 'y', source=source, color='black', line_width=.1)

        return p

    p = make_base_map(map_width=column_width,map_height=500, xaxis=None, yaxis=None,
                    xrange=(-9990000,-9619944), yrange=(5011119,5310000),plot_tools="pan,wheel_zoom,reset,save")

    poly_plot = make_poly_map(p, counties, 'COUNTY','poly_color',.5,None,2,False,"planning")
    line_plot = make_line_map(p, mhn)
    cmap_plot = make_poly_map(p, counties, 'COUNTY',None,.5,'white',2,True,None)


    h_1 = Div(text = """<h1># Overview</h1><hr>
                    <p>Contrary to popular belief, Lorem Ipsum is not simply random text.
                    It has roots in a piece of classical Latin literature from 45 BC,
                    making it over 2000 years old. Richard McClintock, a Latin professor
                    at Hampden-Sydney College in Virginia, looked up one of the more obscure
                    Latin words, consectetur, from a Lorem Ipsum passage, and going through
                    the cites of the word in classical literature, discovered the undoubtable
                    source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus
                    Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in
                    45 BC. This book is a treatise on the theory of ethics, very popular during
                    the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit
                    amet..", comes from a line in section 1.10.32.</p>""",
                    width = column_width, sizing_mode='stretch_both',
                    style={"width":'100%',"text-align":'left',"margin":'0 auto'})

    map_title = Div(text="""<h5>Figure # - CMAP Modeling Area And Network</h5>""",
                    width = column_width, css_classes = ["caption"])


    layout = row(column(h_1,Spacer(height=25),h_2, Spacer(height=25),map_title,cmap_plot,Spacer(height=25),h_3))
    return layout


In [95]:
#read in data

wksp = r'C:\Users\bross\Documents\Python Scripts\ABM\ABM_Report\data\model'

#abm output & input files - individual and join trips
iTripscsv = pd.read_csv(os.path.join((wksp),'outputs\indivTripData_1.csv'),
                        usecols = [u'hh_id',u'person_id',u'tour_id',u'orig_maz',u'dest_maz',
                                   u'trip_mode',
                                   u'orig_purpose',u'dest_purpose',u'board_tap', u'alight_tap'],
                        dtype = {'hh_id': np.int64, 'person_id': np.int64, 'tour_purpose': object,
                                 'trip_mode': np.int64, 'tour_mode': np.int64, 'orig_purpose': object, 'dest_purpose': object,
                                 'board_tap': np.int64, 'alight_tap': np.int64}
                       )
jTrips = pd.read_csv(os.path.join((wksp),'outputs\jointTripData_1.csv'),
                    usecols = [u'hh_id',u'tour_id',u'orig_maz',u'dest_maz',
                               u'trip_mode', 
                               u'orig_purpose',u'dest_purpose',u'num_participants',
                              u'board_tap', u'alight_tap'],
                    dtype = {'hh_id': np.int64,
                            'trip_mode': np.int64,  'orig_purpose': object, 'dest_purpose': object,
                            'num_participants': np.int64,'board_tap': np.int64, 'alight_tap': np.int64}
                    )

hhm = pd.read_csv(os.path.join((wksp),'outputs\hhData_1.csv'),usecols=['hh_id','maz','income','autos',
                                                                    'size','workers','auto_suff'],
                  dtype = {'hh_id': np.int64, 'maz':  np.int64,
                            'income': np.int64, 'autos': np.int64, 'size': np.int64,
                            'workers': np.int64, 'auto_suff': object}
                 )

perm = pd.read_csv(os.path.join((wksp),'outputs\personData_1.csv'),usecols=['hh_id','person_id','person_num',
                                                                           'age','gender','type'],
                  dtype = {'hh_id': np.int64, 'person_id':  np.int64,
                            'age': np.int64, 'type': object}
                  )

iTourscsv = pd.read_csv(os.path.join((wksp),'outputs\indivTourData_1.csv'), 
                        usecols = ['hh_id', 'person_id', 'tour_id',
                                   'tour_category', 'tour_purpose', 'orig_maz',
                                   'dest_maz', 
                                   'tour_mode'],
                         dtype = {'hh_id': np.int64, 'person_id': np.int64, 'tour_id' : np.int64, 'tour_category':str, 
                                  'tour_purpose' : str, 'orig_maz' : np.int64, 'dest_maz' : np.int64, 
                             'tour_mode': object})


jTourscsv = pd.read_csv(os.path.join((wksp),'outputs\jointTourData_1.csv'), 
                        usecols = ['hh_id','tour_id', 'tour_participants',
                                   'tour_purpose', 'orig_maz',
                                   'dest_maz', 
                                   'tour_mode'],
                         dtype = {'hh_id': np.int64, 'tour_id' : np.int64,'tour_participants' : str,
                                  'tour_purpose' : str, 'orig_maz' : np.int64, 'dest_maz' : np.int64, 
                             'tour_mode': object})

In [96]:
ecd = pd.read_csv(r'C:\Users\bross\Documents\GitHub\abm_brown_bag\data\ecd_subzones.csv')

In [102]:
ecd_hh = hhm.merge(ecd,how='left',right_on='subzone09',left_on='maz').dropna()

#outputs
hh_sample = hhm[['hh_id', 'maz', 'income', 'autos', 'size', 'workers', 'auto_suff']].loc[(hhm['income']>0) & \
                                                                                           (hhm['workers']>0)].sample(5)
hh_ids = hh_sample['hh_id'].values.tolist()

per_sample = perm.loc[perm['hh_id'].isin(hh_ids)]
per_ids = per_sample['person_id'].values.tolist()



iTours_sample = iTourscsv.loc[(iTourscsv['hh_id'].isin(hh_ids)) & (iTourscsv['person_id'].isin(per_ids))]
jTours_sample = jTourscsv.loc[(jTourscsv['hh_id'].isin(hh_ids))]


itrips_sample = iTripscsv.loc[(iTripscsv['hh_id'].isin(hh_ids)) & (iTripscsv['person_id'].isin(per_ids))]
jtrips_sample = jTrips.loc[jTrips['hh_id'].isin(hh_ids)]


In [119]:
hh_sample.to_csv(r'C:\Users\bross\Documents\GitHub\abm_brown_bag\data\sample_data\hh_sample.csv')
per_sample.to_csv(r'C:\Users\bross\Documents\GitHub\abm_brown_bag\data\sample_data\per_sample.csv')
iTours_sample.to_csv(r'C:\Users\bross\Documents\GitHub\abm_brown_bag\data\sample_data\itour_sample.csv')
itrips_sample.to_csv(r'C:\Users\bross\Documents\GitHub\abm_brown_bag\data\sample_data\itrips_sample.csv')

In [118]:
### plots with python call backs require the bokeh server
#plots should be inializted from bokeh application handler
column_width = 1000


def bokeh_tab(doc):
    
    
    def load_image(image, title_text):
        p = figure(x_range=(0,250), y_range=(0,410),plot_width=600, plot_height=800,
                   x_axis_location=None, y_axis_location=None,tools='',title=title_text)
        p.image_url(url=[image], x=0, y=1, w=250, h=410, anchor="bottom_left")
        return p
    
    def overview_tab():
        
        image = load_image('data/abm_flow_chart.png','CT-RAMP Structure and Sub-Models')
        overview_text = Div(text="""<h1>Activity Based Model Overview</h1>""", width = column_width)
        ctramp_text = Div(text="""<h2>Coordinated Travel - Regional Activity Modeling Platform 
                            (CT-RAMP) </h2><p>ABM model implements the CT-RAMP design and software platform. 
                            Features microsimulation of travel decisions for individual households and persons 
                            within a household as well as interactions between persons within a household
                            across a range of activity and travel dimensions.</p>
                            <ol><li><b>Population synthesis</b> creates and distributes households and persons
                            for use in the ABM</li>
                            <li><b>Long-Term Location Choice</b> - Models location of usual (mandatory) destinations</li>
                            <li><b>Mobility</b> - Models household attributes like free parking eligibility, car ownership, 
                            transit pass, or toll transponder</li>
                            <li><b>Coordinated Daily Activity-Travel Pattern</b> - Generates and schedules mandatory 
                            and non-mandatory activities for each household member.</li>
                            <li><b>Tour</b> - Daily activities are organized into tours, tour mode, number,
                            and location of stops are determined.</li>
                            <li><b>Trips</b> - Mode, parking, and location of trips making up tour is determined.</li>
                            <li><b>Network Simulation</b> - List of trips for each individual and/or travel party
                            is generated and trips routes are assigned on the modeling network for auto and transit.</li>
                            </ol>""", width = int(column_width*.3))
        extra = Div(text="""<hr><p>Tour: Chain of trips that start and end at home<br>Activities: 10 travel purposes (work, university, school, escorting,
                                shopping, other maintenance, eating out, visiting relatives and friends, 
                                other discretionary, and atwork)</p>""", width = int(column_width*.3),
                               css_classes = ['small'])
        
        return column(Spacer(height=25),overview_text,row(column(ctramp_text,extra),Spacer(width=50),column(image)))
    
    
    def key_features():
        
        kf_title = Div(text="""<h1>ABM Features</h1>""", width = column_width)
        
        kf_tours = Div(text="""<h2>Travel Is Organized Into Tours and Trips</h2>
                           """, width = column_width)
        
        kf_traveler = Div(text="""<h2>Each Traveler Can Be Identified Throughout Similuation</h2>
        <p>As individual travelers are simulated the results are listed by household, person, tour, and trip.
        Allows trip/tour results to be represented across any number of categories included in the synthetic population results.</p>
                           """, width = column_width)
        
        kf_attr = Div(text="""<h2>Each Traveler’s Personal Attributes Inform Unique Travel Choices</h2>
        <p>Person and household decision making units such as person type, age, work/school status, income, available vehicles
            inform travel choices.</p>
                           """, width = column_width)
        
        return column(Spacer(height=25),kf_title, kf_tours,kf_traveler,kf_attr)
    
    def output_tab():
    
        hh_col = [TableColumn(field=col, title=col) for col in hh_sample.columns[:2]]+\
                 [TableColumn(field='income', title='income',formatter=NumberFormatter(format="$0,0.00"))]+\
                 [TableColumn(field=col, title=col) for col in hh_sample.columns[3:]]
        
        per_col = [TableColumn(field=col, title=col) for col in per_sample.columns]
        tour_col = [TableColumn(field=col, title=col) for col in iTours_sample.columns]
        trip_col = [TableColumn(field=col, title=col) for col in itrips_sample.columns]

        hh_src = ColumnDataSource(hh_sample.sort_values(by='hh_id'))
        per_src = ColumnDataSource(per_sample.sort_values(by='hh_id'))
        tour_src = ColumnDataSource(iTours_sample.sort_values(by='person_id'))
        trip_src = ColumnDataSource(itrips_sample.sort_values(by='person_id'))


        hh_div = Div(text="""<h2>Household Attribution Results</h2><p>Individual household attributes</p>
                            <ul><li><b>hh_id</b> : Household ID</li>
                            <li><b>maz</b> : Household Subzone</li>
                            <li><b>income</b> : Household Income</li>
                            <li><b>autos</b> : Number of Vehicles</li>
                            <li><b>size</b> : Household Size</li>
                            <li><b>workers</b> : Number of Workers in Household</li>
                            <li><b>auto_suff</b> : Auto Sufficiency</li></ul>""", width = column_width)
        hh_tbl = DataTable(columns=hh_col, source=hh_src, height = 200, selectable = True,width=int(column_width*.75),
                                 fit_columns = True, scroll_to_selection=True)

        per_div = Div(text="""<h2>Person Attribution Results</h2><p>Individual persons within a household</p>
                            <ul><li><b>hh_id</b> : Household ID</li>
                            <li><b>person_id</b> : Person ID</li>
                            <li><b>per_num</b> : Person Number in Household</li>
                            <li><b>age</b> : Age</li>
                            <li><b>gender</b> : Gender</li>
                            <li><b>type</b> : Person Type (worker, student, etc)</li></ul>""", width = column_width)
        per_tbl = DataTable(columns=per_col, source=per_src,height = 200,selectable = True,width=int(column_width*.75),
                             fit_columns = True, scroll_to_selection=True)
        
        tour_div = Div(text="""<h2>Individual Tour Results</h2><p>Individual tours by person and household</p>
                            <ul><li><b>hh_id</b> : Household ID</li>
                            <li><b>person_id</b> : Person ID</li>
                            <li><b>tour_id</b> : Tour ID (0=first tour, 1 second tour, ect)</li>
                            <li><b>tour_category</b> : Mandatory, Non-Mandatory</li>
                            <li><b>tour_purpose</b> : Purpose of travel</li>
                            <li><b>maz</b> : Origin and destination subzone</li>
                            <li><b>tour_mode</b> : Mode of travel</li></ul>""", width = column_width)
        tour_tbl = DataTable(columns=tour_col, source=tour_src,height = 250,selectable = True,width=int(column_width*.75),
                             fit_columns = True, scroll_to_selection=True)
        
        
        trip_div = Div(text="""<h2>Individual Trip Results</h2><p>Individual tours by person and household</p>
                            <ul><li><b>hh_id</b> : Household ID</li>
                            <li><b>person_id</b> : Person ID</li>
                            <li><b>tour_id</b> : Tour ID (0=first tour, 1 second tour, ect)</li>
                            <li><b>purpose</b> : Origin and destination trip purpose</li>
                            <li><b>maz</b> : Origin and destination subzone</li>
                            <li><b>trip_mode</b> : Mode of travel</li>
                            <li><b>tap</b> : boarding and alighting transit id</li></ul>""", width = column_width)
        trip_tbl = DataTable(columns=trip_col, source=trip_src,height = 250,selectable = True,width=int(column_width*.75),
                             fit_columns = True, scroll_to_selection=True)

        def hh_select():

            indices = hh_src.selected.indices
            if len(indices) == 1:
                hh_id = hh_src.data["hh_id"][indices[0]]

                new_indices = [i for i, h in enumerate(per_src.data["hh_id"]) if h == hh_id]
                per_src.selected.indices=new_indices


        #hh_src.on_change('selected',my_tap_handler)

        per_button = Button(label="Select Person Level", button_type="default")
        per_button.on_click(hh_select)
        
        
        def per_select():

            indices = per_src.selected.indices
            if len(indices) == 1:
                per_id = per_src.data["person_id"][indices[0]]

                new_indices = [i for i, p in enumerate(tour_src.data["person_id"]) if p == per_id]
                tour_src.selected.indices=new_indices


        tour_button = Button(label="Select Person Tour", button_type="default")
        tour_button.on_click(per_select)
        

        def trip_select():

            indices = tour_src.selected.indices
            if len(indices) == 1:
                per_id = tour_src.data["person_id"][indices[0]]
                tour_id = tour_src.data["tour_id"][indices[0]]
                
                
                new_indices = []
                i = 0
                while i < len(trip_src.data["person_id"]):
                    #trip_src.data["person_id"][i] == per_id
                    if trip_src.data["person_id"][i] == per_id:
                        if trip_src.data["tour_id"][i] == tour_id:
                            new_indices.append(i)
                    i+=1

                trip_src.selected.indices=new_indices
                
        trip_button = Button(label="Select Person Trip", button_type="default")
        trip_button.on_click(trip_select)

        output_title = Div(text = """<h1>Model Output Files</h1>""",width=column_width)
        output_description = Div(text = """<p>The ABM produces output files for modeled householdes, modeled persons, 
        mandatory trip locations (work, school, university), trips, tours, ect. Model data calibrated to the 
        CMAP Household Travel Survey (2007-2009)<p>""",width=column_width)
        
        return column(Spacer(height=25),output_title,output_description, hh_div, hh_tbl,  per_div, per_button,per_tbl,
                     Spacer(height=10),tour_div,tour_button, tour_tbl, Spacer(height=10),trip_div,trip_button, trip_tbl)
    
    
    
    h_1 = Div(text = """<h1><center>Intro Text</center></h1>""",width=column_width)
    h_2 = Div(text = """<h1><center>Intro Text</center></h1>""")
    h_4 = Div(text = """<h1><center>Intro Text</center></h1>""")

    l_1 = layout(children=[overview_tab()])
    l_2 = layout(children=[key_features()])
    l_3 = layout(children=[output_tab()])
    l_4 = layout(children=[h_4])

    tab_1 = Panel(child=l_1, title = 'Overview')
    tab_2 = Panel(child=l_3, title = 'Outputs')
    tab_3 = Panel(child=l_2, title = 'Features')
    tab_4 = Panel(child=l_4, title = 'Data Exploration')
    
    tabs = Tabs(tabs = [tab_1, tab_2, tab_3, tab_4], sizing_mode = "stretch_both")
    
    doc.add_root(tabs)
    
    
handler = FunctionHandler(bokeh_tab)
app = Application(handler)
show(app)

